## 一、双指针算法
for (int i = 0, j = 0; i < n; i ++ )
{
    while (j < i && check(i, j)) j ++ ;
    // 具体问题的逻辑
}
常见问题分类：
    (1) 对于一个序列，用两个指针维护一段区间
    (2) 对于两个序列，维护某种次序，比如归并排序中合并两个有序序列的操作
双指针算法将暴力循环的时间复杂度优化到O（n）

In [1]:
# AcWing 799. 最长连续不重复子序列
n1 = int(input())
data = [int(x) for x in input().split()]
res = 0
s = [0] * 100010

def f1():  # 朴素法
    global res

    def check(l,r):
        for i in range(l+1,r+1):
            j = l
            while j < i:
                if data[i] == data[j]:
                    return 1
                j += 1
        return 0

    for i in range(n1):
        j = 0
        while j < i:
            # 得到某个连续子区间
            if check(j, i) == 0:
                res = max(i-j+1,res)
            j += 1
    print(res)

def f2():  # 双指针做法 j是不会后退的，即：j具有单调性
    global res

    def check(l,r):
        for i in range(l+1,r+1):
            j = l
            while j < i:
                if data[i] == data[j]:
                    return 1
                j += 1
        return 0

    j = 0
    for i in range(n1):
        while j < i:
            if check(j,i) == 0:
                res = max(res,i-j+1)
                break
            j += 1
    print(res)

def f3():  # 优化check
    global res
    j = 0
    for i in range(n1):
        s[data[i]] += 1          # 当前字符出现的次数+1
        while s[data[i]] > 1:    # 如果当前字符出现次数>1,表示有重复，需要进行处理
            s[data[j]] -= 1      # 让j开始往右走，走一次，看一次是否还有重复
            j += 1
        res = max(res, i-j+1)  # 更新答案

    print(res)

3


## 二、 位运算
### 1. n的二进制表示中第k位是几
n= 15 =(1111)2  个位是第0位
①先把第k位移到最后一位 n >> k
②看个位是几 x & 1
### 2. lowbit(x):返回x的最后一位1
树状数组基础
x = 1010 lowbit(x)=10
x = 101000 lowbit(x)=1000
实现：x & -x  (-x = ~x+1)
x    = 1010...1000
~x   = 0101...0111
~x+1 = 0101...1000
x&-x = 0000...1000
补充知识：
原码：是最简单的机器数表示法。用最高位表示符号位，‘1’表示负号，‘0’表示正号。其他位存放该数的二进制的绝对值。
所以1010表示十进制数（-2），同理 0010表示十进制数（2）
反码：正数的反码还是等于原码，负数的反码就是他的原码除符号位外，按位取反。
补码：正数的补码等于他的原码， 负数的补码等于反码+1。也等于正数反码+1。
原码：0...01010
反码：1...10101
补码：1...10110 (~x+1

In [1]:
# AcWing 801. 二进制中1的个数
def lowbit(x):
    return x & -x

def main():
    n = int(input())
    x = list(map(int,input().split()))
    for i in x:
        res = 0
        while i != 0:
            i -= lowbit(i)
            res += 1
        print(res,end=' ')

main()

1 1 2 1 2 

## 三、离散化
值域0~10^9  个数10^5
a[]:1, 3, 100, 2000, 500000
↓映射
    0, 1, 2, 3, 4
①a中可能有重复元素——去重
②如何算出a中每个元素映射后的值——二分

In [2]:
# AcWing 802. 区间和
def find(x, alls):
    # 给定离散前的值，找出离散后的值
    # 二分查找，找出>=x的第一个数
    l = 0
    r = len(alls) - 1
    while l < r:
        mid = (l+r) // 2
        if alls[mid] >= x:
            r = mid
        else:
            l = mid + 1
    return l + 1

def main():
    N = 300010   # 最多要用到三十万个坐标：插入(x),查询(l,r)

    n,m = map(int,input().split())
    a = [0] * N  # 离散索引：离散前的坐标所拥有值
    s = [0] * (N+1)  # 前缀和数组
    alls = [] # 所有使用过的坐标
    add = [] # 二元组，(插入值,插入坐标)
    query = [] # 二元组，(l,r)
    # 收集插入操作
    for i in range(n):
        x, c = map(int,input().split())
        add.append((x,c))
        alls.append(x)

    # 收集询问操作
    for i in range(m):
        l, r = map(int,input().split())
        query.append((l,r))
        alls.append(l)
        alls.append(r)

    # 对alls进行去重
    alls = list(set(alls))

    # 对离散后的数组进行插入操作
    for x,c in add:
        x2 = find(x,alls)  # 离散数组中的位置
        a[x2-1] += c

    # 求前缀和
    for i in range(1,N+1):
        s[i] = s[i-1] + a[i-1]

    # 进行m次询问
    for l,r in query:
        xl = find(l,alls)
        xr = find(r,alls)
        print(s[xr] - s[xl-1])

main()

8
0
5


## 四、区间合并
①按区间左端点排序
②维护一段区间，左端点st，右端点ed，如果两个区间之间有交集(第一个区间的尾部 >= 第二个区间的头部)，
则合并，更新尾部end = max(end, 第二个区间和第一个区间哪个尾部值最大)
否则重新标记新的区间头start和尾部end为第二个区间的头和尾

In [ ]:
def main():
    n = int(input())
    segs = []
    # 读入
    for _ in range(n):
        l,r = map(int,input().split())
        segs.append((l,r))
    def merge(segs):
        res = []
        segs.sort(key=lambda x:x[0])
        st, ed = -2e9, -2e9
        for seg in segs:
            if ed < seg[0]:  # 找到新的区间
                if st != -2e9:
                    res.append((st,ed))
                st,ed = segs[0],segs[1]
            else:  # 合并区间
                ed = max(ed,seg[1])
        if st != 2e-9:
            res.append((st,ed))
        return res
    segs = merge(segs)
    print(len(segs))